In [ ]:
!pip install rasterio -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 96.6 MB/s eta 0:00:00


In [ ]:
# Bibliotecas
import numpy as np
import rasterio
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [ ]:
# Caminhos dos arquivos
caminho_cubo = "/content/drive/MyDrive/Dados_PDI/cubo_DF_S1.tif"
caminho_roi = "/content/drive/MyDrive/Dados_PDI/roi_DF_S1.tif"

# Carregar o cubo multitemporal
with rasterio.open(caminho_cubo) as src:
    cubo = src.read()

# Carregar as amostras de treinamento (ROI)
with rasterio.open(caminho_roi) as src:
    roi = src.read(1)

In [ ]:
# Transpor cubo para (largura, altura, n_tempos)
cubo = np.moveaxis(cubo, 0, -1)

# Identificar pixels rotulados no ROI (excluir pixels sem rótulo, geralmente 0 ou -1)
mask = roi > 0
X_train = cubo[mask]
y_train = roi[mask]
y_train -= 1

In [ ]:
# Definir a forma dos dados
n_tempos = X_train.shape[1]  # Número de imagens ao longo do tempo
n_bandas = 1 if len(X_train.shape) == 2 else X_train.shape[2]

# Reshape necessário se tivermos apenas uma banda
if n_bandas == 1:
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)

# Criar o modelo LSTM
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(n_tempos, n_bandas)),
    Dropout(0.3),
    LSTM(32, return_sequences=False),
    Dropout(0.3),
    Dense(16, activation='relu'),
    Dense(len(np.unique(y_train)), activation='softmax')
])

# Compilar o modelo
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Treinar
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

In [ ]:
# Se o cubo tem múltiplas bandas, manter a forma correta
if n_bandas == 1:
    cubo = cubo.reshape(cubo.shape[0], cubo.shape[1], cubo.shape[2], 1)

# Aplicar modelo para classificar cada pixel
cubo_reshape = cubo.reshape(-1, n_tempos, n_bandas)
preds = model.predict(cubo_reshape)
preds = np.argmax(preds, axis=1)

# Reformatar para imagem original
classificacao = preds.reshape(cubo.shape[0], cubo.shape[1])

In [ ]:
# Salvar como novo raster
with rasterio.open(
    "/content/drive/MyDrive/Dados_PDI/classificacao_LSTM.tif",
    "w",
    driver="GTiff",
    height=classificacao.shape[0],
    width=classificacao.shape[1],
    count=1,
    dtype=rasterio.uint8,
    crs=src.crs,
    transform=src.transform
) as dst:
    dst.write(classificacao, 1)